In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: arbitraryyy, just to make sure after filtered, it's not empty
dummy_data = np.round(np.random.uniform(1,100,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
gt30_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 30].reshape(1,-1,1)
dummy_theory_output = torch.sqrt(torch.var(gt30_dummy_data_tensor, correction = 1))
dummy_data_mean = torch.mean(gt30_dummy_data_tensor)

In [6]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = dummy_data_mean, requires_grad = False)

    def forward(self,X):
        num_fil_X = torch.sum((X>30).double())
        fil_mean_X = torch.where(X>30, X, 0.0)
        x_mean_cons = torch.abs(torch.sum(fil_mean_X)-num_fil_X*(self.data_mean))<=torch.abs(0.01*num_fil_X*self.data_mean)
        fil_std_X = torch.where(X>30, X, self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((fil_std_X-self.data_mean)*(fil_std_X-self.data_mean))-self.w*self.w*(num_fil_X-1))<=torch.abs(0.02*self.w*self.w*(num_fil_X-1)),x_mean_cons),self.w)
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [7]:
# Prover/ data owner side
gt30_data_tensor = data_tensor[data_tensor > 30].reshape(1,-1,1)
theory_output = torch.sqrt(torch.var(gt30_data_tensor, correction = 1))
data_mean = torch.mean(gt30_data_tensor)
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = data_mean, requires_grad = False)
        
    def forward(self,X):
        num_fil_X = torch.sum((X>30).double())
        fil_mean_X = torch.where(X>30, X, 0.0)
        x_mean_cons = torch.abs(torch.sum(fil_mean_X)-num_fil_X*(self.data_mean))<=torch.abs(0.01*num_fil_X*self.data_mean)
        fil_std_X = torch.where(X>30, X, self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((fil_std_X-self.data_mean)*(fil_std_X-self.data_mean))-self.w*self.w*(num_fil_X-1))<=torch.abs(0.02*self.w*self.w*(num_fil_X-1)),x_mean_cons),self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [2], "resources", settings_path)

theory output:  tensor(12.7586)
==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-161232,28184],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":5734,"total_const_size":612,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/utils.py:1686: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you believe there is a bug, please report an issue at https://github.com/pytorch/pytorch/issues. Error reported by strict ONNX shape inference: [ShapeInferenceError] (op_type:Where, node name: /Where_1): Y has inconsistent type tensor(float) (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/jit/serialization/export.cpp:1421.)
  _C._check_onnx_proto(proto)


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 30.583266735076904 seconds
Theory output:  tensor(12.7586)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 12.75
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[10537101196673941533, 17227541574925932677, 11187715152301828262, 1869164017182098189], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12362648763242643187, 13940026059969050459, 6027715406760125980, 2781413989188023337]]], 'proof': '14a070362d5a0ccd7215fe638a99200ba2b8a11b08cf595e5125559696bb22b41ffe4e57a8c61fd7e58aaaabce9229bcf7c55b82e0c351895a68a21476b3ea912d706d3205a719b5f8550e5bf9e6acf88e8b85d5764f2f83e5616f00c2a24db31b19c36683be0dab0aad98685e00308829785f2a1a6928d91d606ca358df02431ecdf41aed0a3c9496e7a5127ee6b6fefb16d093b49918315505a0f51d20e0d12208e154a11f551377e237af305510d33c5b89c3e5cb0cca63a9bcba223b16cc28b24fe083e451ddec941f08eba1d734a30a438ecc0af38d00df0ac53cf68dc629d97f9a7cc5c5ba3b3b67f8b6f83a5fd978835725039eebc2cdff06fe88e8a02e8d9c9e04e2455f295c8eda5091185dcaa3225ef70079d6a7bf4d39a74f5dec1b16c0a746522ecce4395df34dff9a73e160109d474889dfbcdb936c0d35e0a32201a06b48282dae5091f926f0d5a5c026295a1d74f8167f9473a2a24d11

In [9]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[10537101196673941533, 17227541574925932677, 11187715152301828262, 1869164017182098189], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12362648763242643187, 13940026059969050459, 6027715406760125980, 2781413989188023337]]]
proof boolean:  1.0
proof result 1 : 12.75
verified
